
## Advanced RAG Techniques!


In [21]:
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from chromadb import PersistentClient
from tqdm import tqdm
from litellm import completion
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go


load_dotenv(override=True)

MODEL = "gpt-4.1-nano"

DB_NAME = "preprocessed_db"
collection_name = "docs"
embedding_model = "text-embedding-3-large"
# embedding_model = "all-MiniLM-L6-v2"
KNOWLEDGE_BASE_PATH = Path("knowledge-base")
AVERAGE_CHUNK_SIZE = 500

openai = OpenAI()

In [2]:
# Inspired by LangChain's Document Object

class Result(BaseModel):
    page_content: str
    metadata: dict

In [5]:
# A class to perfectly represent a chunk

class Chunk(BaseModel):
    headline: str = Field(description="A brief heading for this chunk, typically a few words, that is most likely to be surfaced in a query")
    summary: str = Field(description="A few sentences summarizing the content of this chunk to answer common questions")
    original_text: str = Field(description="The original text of this chunk from the provided document, exactly as is, not changed in any way")

    def as_result(self, document):
        metadata = {"source": document["source"], "type": document["type"]}
        return Result(page_content=self.headline + "\n\n" + self.summary + "\n\n" + self.original_text,metadata=metadata)


class Chunks(BaseModel):
    chunks: list[Chunk]

### STEPS
1. Fetch documents from the knowledge base, like LangChain did
2. Call an LLM to turn documents into Chunks
3. Store the Chunks in Chroma


In [6]:
def fetch_documents():
    """A homemade version of the LangChain DirectoryLoader"""

    documents = []

    for folder in KNOWLEDGE_BASE_PATH.iterdir():
        doc_type = folder.name
        for file in folder.rglob("*.md"):
            with open(file, "r", encoding="utf-8") as f:
                documents.append({"type": doc_type, "source": file.as_posix(), "text": f.read()})

    print(f"Loaded {len(documents)} documents")
    return documents

In [7]:
documents = fetch_documents()

Loaded 76 documents


In [8]:
documents[0]

{'type': 'company',
 'source': 'knowledge-base/company/about.md',
 'text': "# About Insurellm\n\nInsurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. Its first product was Markellm, the marketplace connecting consumers with insurance providers.\n\nThe company experienced rapid growth in its first five years, expanding its product portfolio to include Carllm (auto insurance portal), Homellm (home insurance portal), and Rellm (enterprise reinsurance platform). By 2020, Insurellm had reached a peak of 200 employees with 12 offices across the US.\n\nHowever, the company underwent a strategic restructuring in 2022-2023 to focus on profitability and sustainable growth. This included consolidating office locations, implementing a remote-first strategy, and streamlining operations. As of 2025, Insurellm operates with a lean, highly efficient team of 32 employees who have built a portfolio of 32 active cont

### STEP 2: Making the prompt that will help us make more chunks

Use LLM to Help Create relevant Chunks




In [9]:
def make_prompt(document):
    how_many = (len(document["text"]) // AVERAGE_CHUNK_SIZE) + 1
    return f"""
You take a document and you split the document into overlapping chunks for a KnowledgeBase.

The document is from the shared drive of a company called Insurellm.
The document is of type: {document["type"]}
The document has been retrieved from: {document["source"]}

A chatbot will use these chunks to answer questions about the company.
You should divide up the document as you see fit, being sure that the entire document is returned in the chunks - don't leave anything out.
This document should probably be split into {how_many} chunks, but you can have more or less as appropriate.
There should be overlap between the chunks as appropriate; typically about 25% overlap or about 50 words, so you have the same text in multiple chunks for best retrieval results.

For each chunk, you should provide a headline, a summary, and the original text of the chunk.
Together your chunks should represent the entire document with overlap.

Here is the document:

{document["text"]}

Respond with the chunks.
"""

In [10]:
print(make_prompt(documents[0]))


You take a document and you split the document into overlapping chunks for a KnowledgeBase.

The document is from the shared drive of a company called Insurellm.
The document is of type: company
The document has been retrieved from: knowledge-base/company/about.md

A chatbot will use these chunks to answer questions about the company.
You should divide up the document as you see fit, being sure that the entire document is returned in the chunks - don't leave anything out.
This document should probably be split into 5 chunks, but you can have more or less as appropriate.
There should be overlap between the chunks as appropriate; typically about 25% overlap or about 50 words, so you have the same text in multiple chunks for best retrieval results.

For each chunk, you should provide a headline, a summary, and the original text of the chunk.
Together your chunks should represent the entire document with overlap.

Here is the document:

# About Insurellm

Insurellm was founded by Avery La

In [11]:
def make_messages(document):
    return [
        {"role": "user", "content": make_prompt(document)},
    ]

In [14]:
def process_document(document):
    messages = make_messages(document)
    response = completion(model=MODEL, messages=messages, response_format=Chunks)
    reply = response.choices[0].message.content
    print(reply)
    doc_as_chunks = Chunks.model_validate_json(reply).chunks
    print(doc_as_chunks)
    return [chunk.as_result(document) for chunk in doc_as_chunks]

In [15]:
process_document(documents[0])


{"chunks":[{"headline":"Introduction to Insurellm","summary":"Insurellm was founded in 2015 by Avery Lancaster as an innovative insurance technology startup, initially offering a marketplace connecting consumers with insurance providers.","original_text":"Insurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. Its first product was Markellm, the marketplace connecting consumers with insurance providers."},{"headline":"Company Growth and Product Expansion (2015-2020)","summary":"The company experienced rapid growth, expanding its product portfolio to include auto, home, and reinsurance portals, reaching 200 employees and 12 offices across the US by 2020.","original_text":"The company experienced rapid growth in its first five years, expanding its product portfolio to include Carllm (auto insurance portal), Homellm (home insurance portal), and Rellm (enterprise reinsurance platform). By 2020, Insurellm 

[Result(page_content='Introduction to Insurellm\n\nInsurellm was founded in 2015 by Avery Lancaster as an innovative insurance technology startup, initially offering a marketplace connecting consumers with insurance providers.\n\nInsurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products. Its first product was Markellm, the marketplace connecting consumers with insurance providers.', metadata={'source': 'knowledge-base/company/about.md', 'type': 'company'}),
 Result(page_content='Company Growth and Product Expansion (2015-2020)\n\nThe company experienced rapid growth, expanding its product portfolio to include auto, home, and reinsurance portals, reaching 200 employees and 12 offices across the US by 2020.\n\nThe company experienced rapid growth in its first five years, expanding its product portfolio to include Carllm (auto insurance portal), Homellm (home insurance portal), and Rellm (enterprise reinsur

In [16]:
def create_chunks(documents):
    chunks = []
    for doc in tqdm(documents):
        chunks.extend(process_document(doc))
    return chunks

In [17]:
chunks = create_chunks(documents)

  1%|▏         | 1/76 [00:06<08:09,  6.52s/it]

{"chunks":[{"headline":"Introduction and Founding of Insurellm","summary":"Insurellm was established by Avery Lancaster in 2015 as an innovative insurance technology startup aiming to disrupt the industry with new products.","original_text":"Insurellm was founded by Avery Lancaster in 2015 as an insurance tech startup designed to disrupt an industry in need of innovative products."},{"headline":"Early Product Portfolio and Initial Growth","summary":"The company's first product was Markellm, connecting consumers with insurance providers, leading to rapid expansion and additional portals for auto, home, and reinsurance.","original_text":"Its first product was Markellm, the marketplace connecting consumers with insurance providers.\n\nThe company experienced rapid growth in its first five years, expanding its product portfolio to include Carllm (auto insurance portal), Homellm (home insurance portal), and Rellm (enterprise reinsurance platform). By 2020, Insurellm had reached a peak of 20

  3%|▎         | 2/76 [00:19<13:03, 10.58s/it]

{"chunks":[{"headline":"Introduction: Careers at Insurellm","summary":"An overview of Insurellm's mission, growth, and team culture highlighting the company's focus on innovation, client impact, and diverse values.","original_text":"# Careers at Insurellm\n\n## Why Join Insurellm?\n\nAt Insurellm, we're not just building software—we're revolutionizing an entire industry. Since our founding in 2015, we've evolved from a high-growth startup to a lean, profitable company with 32 highly talented employees managing 32 active contracts across all eight of our product lines.\n\nAfter reaching 200 employees in 2020, we strategically restructured in 2022-2023 to focus on sustainable growth, operational excellence, and building a world-class remote-first culture. Today, we're a tight-knit team of exceptional professionals who deliver outsized impact through automation, AI, and strategic focus on high-value enterprise clients—from regional insurers to global reinsurance partners.\n\n### Our Cultu

  4%|▍         | 3/76 [00:27<10:55,  8.97s/it]

...

{"chunks":[{"headline":"Product Overview of Lifellm","summary":"Lifellm is an advanced life insurance platform by Insurellm that leverages AI and predictive analytics to offer personalized policies, streamline underwriting, and enhance customer experience across B2B and B2C markets.","original_text":"# Product Summary\n\n# Lifellm\n\n## Summary\n\nLifellm is an advanced life insurance platform developed by Insurellm that revolutionizes how insurance providers offer life coverage to their customers. Leveraging cutting-edge AI and predictive analytics, Lifellm enables insurers to provide personalized life insurance policies with streamlined underwriting, accurate risk assessment, and exceptional customer experiences. Designed for both B2B and B2C markets, Lifellm helps insurance companies modernize their life insurance operations while reducing costs and improving customer satisfaction."},{"headline":"Lifellm Features: AI-Powered Underwriting and Predictive Risk Modeling","summary":"Life

 99%|█████████▊| 75/76 [11:17<00:08,  8.59s/it]

{"chunks":[{"headline":"Product Overview","summary":"Markellm is an innovative two-sided insurance marketplace powered by AI, designed to connect consumers with insurance providers efficiently and personalize the insurance shopping experience. It offers a user-friendly platform that enhances decision-making and saves time for users.","original_text":"# Product Summary\n\n# Markellm\n\n## Summary\n\nMarkellm is an innovative two-sided marketplace designed to seamlessly connect consumers with insurance companies. Powered by advanced matching AI, Markellm transforms the insurance shopping experience, making it more efficient, personalized, and accessible. Whether you're a homeowner searching for the best rates on home insurance or an insurer looking to reach new customers, Markellm acts as the ultimate bridge, delivering tailored solutions for all parties involved. With a user-friendly interface and powerful algorithms, Markellm not only saves time but also enhances decision-making in the

100%|██████████| 76/76 [11:24<00:00,  9.01s/it]

{"chunks":[{"headline":"Product Overview of Rellm","summary":"Rellm is an AI-enhanced enterprise reinsurance platform by Insurellm designed to improve risk management, decision-making, and operational efficiency with seamless integration and analytics.","original_text":"# Product Summary\n\n# Rellm: AI-Powered Enterprise Reinsurance Solution\n\n## Summary\n\nRellm is an innovative enterprise reinsurance product developed by Insurellm, designed to transform the way reinsurance companies operate. Harnessing the power of artificial intelligence, Rellm offers an advanced platform that redefines risk management, enhances decision-making processes, and optimizes operational efficiencies within the reinsurance industry. With seamless integrations and robust analytics, Rellm enables insurers to proactively manage their portfolios and respond to market dynamics with agility."},{"headline":"Key Features of Rellm","summary":"Rellm's main features include AI-driven analytics, seamless system integ

In [18]:
print(len(chunks))

360


### Create Embeddings

In [19]:
def create_embeddings(chunks):
    chroma = PersistentClient(path=DB_NAME)
    if collection_name in [c.name for c in chroma.list_collections()]:
        chroma.delete_collection(collection_name)

    texts = [chunk.page_content for chunk in chunks]
    emb = openai.embeddings.create(model=embedding_model, input=texts).data
    vectors = [e.embedding for e in emb]

    collection = chroma.get_or_create_collection(collection_name)

    ids = [str(i) for i in range(len(chunks))]
    metas = [chunk.metadata for chunk in chunks]

    collection.add(ids=ids, embeddings=vectors, documents=texts, metadatas=metas)
    print(f"Vectorstore created with {collection.count()} documents")

In [22]:
create_embeddings(chunks)

Vectorstore created with 360 documents


### Visualization

In [23]:
chroma = PersistentClient(path=DB_NAME)
collection = chroma.get_or_create_collection(collection_name)
result = collection.get(include=['embeddings', 'documents', 'metadatas'])
vectors = np.array(result['embeddings'])
documents = result['documents']
metadatas = result['metadatas']
doc_types = [metadata['type'] for metadata in metadatas]
colors = [['blue', 'green', 'red', 'orange'][['products', 'employees', 'contracts', 'company'].index(t)] for t in doc_types]

In [24]:
tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(title='2D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x',yaxis_title='y'),
    width=800,
    height=600,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [25]:
tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=5, color=colors, opacity=0.8),
    text=[f"Type: {t}<br>Text: {d[:100]}..." for t, d in zip(doc_types, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=900,
    height=700,
    margin=dict(r=10, b=10, l=10, t=40)
)

fig.show()

### Applying Advanced RAG Techiniques

1. Reranking - reorder the rank results
2. Query re-writing

In [26]:

class RankOrder(BaseModel):
    order: list[int] = Field(
        description="The order of relevance of chunks, from most relevant to least relevant, by chunk id number"
    )

In [27]:
def rerank(question, chunks):
    system_prompt = """
You are a document re-ranker.
You are provided with a question and a list of relevant chunks of text from a query of a knowledge base.
The chunks are provided in the order they were retrieved; this should be approximately ordered by relevance, but you may be able to improve on that.
You must rank order the provided chunks by relevance to the question, with the most relevant chunk first.
Reply only with the list of ranked chunk ids, nothing else. Include all the chunk ids you are provided with, reranked.
"""
    user_prompt = f"The user has asked the following question:\n\n{question}\n\nOrder all the chunks of text by relevance to the question, from most relevant to least relevant. Include all the chunk ids you are provided with, reranked.\n\n"
    user_prompt += "Here are the chunks:\n\n"
    for index, chunk in enumerate(chunks):
        user_prompt += f"# CHUNK ID: {index + 1}:\n\n{chunk.page_content}\n\n"
    user_prompt += "Reply only with the list of ranked chunk ids, nothing else."
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]
    response = completion(model=MODEL, messages=messages, response_format=RankOrder)
    reply = response.choices[0].message.content
    order = RankOrder.model_validate_json(reply).order
    print(order)
    return [chunks[i - 1] for i in order]

In [29]:
RETRIEVAL_K = 10

def fetch_context_unranked(question):
    query = openai.embeddings.create(model=embedding_model, input=[question]).data[0].embedding
    results = collection.query(query_embeddings=[query], n_results=RETRIEVAL_K)
    chunks = []
    for result in zip(results["documents"][0], results["metadatas"][0]):
        chunks.append(Result(page_content=result[0], metadata=result[1]))
    return chunks

In [30]:
question = "Who won the IIOTY award?"
chunks = fetch_context_unranked(question)

In [31]:
for chunk in chunks:
    print(chunk.page_content[:15]+"...")

Annual Performa...
Career Progress...
Performance His...
Performance and...
Performance His...
Annual Performa...
Annual Performa...
Career Progress...
Performance Rat...
Insurellm Caree...


In [32]:
reranked = rerank(question, chunks)

[4, 5, 9, 3, 8, 6, 7, 10, 2, 1]


In [33]:
for chunk in reranked:
    print(chunk.page_content[:15]+"...")

Performance and...
Performance His...
Performance Rat...
Performance His...
Career Progress...
Annual Performa...
Annual Performa...
Insurellm Caree...
Career Progress...
Annual Performa...


In [34]:
question = "Who went to Manchester University?"
RETRIEVAL_K = 20
chunks = fetch_context_unranked(question)
for index, c in enumerate(chunks):
    if "manchester" in c.page_content.lower():
        print(index)

In [35]:
reranked = rerank(question, chunks)

[8, 6, 14, 4, 11, 12, 16, 10, 13, 17, 18, 19, 20, 1, 2, 3, 5, 7, 9, 15]


In [36]:
for index, c in enumerate(reranked):
    if "manchester" in c.page_content.lower():
        print(index)

The above is not fetching chunks with relevant information first so the reranking also wont' matter. The solution is to use Query Rewriting techinique

In [37]:
reranked[0].page_content

'HR Record Overview for Marcus Johnson\n\nThis section introduces Marcus Johnson, detailing his HR record including education, certifications, recognition, and mentorship contributions.\n\n# HR Record\n\n# Marcus Johnson\n\n## Summary\n- **Date of Birth:** February 17, 1988\n- **Job Title:** Customer Success Manager\n- **Location:** New York, New York\n- **Current Salary:** $98,000\n\n## Insurellm Career Progression\n- **April 2020 - Present:** Customer Success Manager\n  - Manages portfolio of 25 enterprise clients generating $5M in annual revenue\n  - Achieved 95% client retention rate over past three years\n  - Leads quarterly business reviews and strategic planning sessions with clients\n\n- **September 2018 - March 2020:** Customer Success Associate\n  - Supported senior CSMs with client onboarding and training\n  - Managed day-to-day communication with mid-market clients\n  - Created customer success playbooks and best practices documentation\n\n- **January 2016 - August 2018:** 

In [38]:
def fetch_context(question):
    chunks = fetch_context_unranked(question)
    return rerank(question, chunks)

In [39]:
SYSTEM_PROMPT = """
You are a knowledgeable, friendly assistant representing the company Insurellm.
You are chatting with a user about Insurellm.
Your answer will be evaluated for accuracy, relevance and completeness, so make sure it only answers the question and fully answers it.
If you don't know the answer, say so.
For context, here are specific extracts from the Knowledge Base that might be directly relevant to the user's question:
{context}

With this context, please answer the user's question. Be accurate, relevant and complete.
"""

In [40]:
def make_rag_messages(question, history, chunks):
    context = "\n\n".join(f"Extract from {chunk.metadata['source']}:\n{chunk.page_content}" for chunk in chunks)
    system_prompt = SYSTEM_PROMPT.format(context=context)
    return [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": question}]

In [41]:
def rewrite_query(question, history=[]):
    """Rewrite the user's question to be a more specific question that is more likely to surface relevant content in the Knowledge Base."""
    message = f"""
You are in a conversation with a user, answering questions about the company Insurellm.
You are about to look up information in a Knowledge Base to answer the user's question.

This is the history of your conversation so far with the user:
{history}

And this is the user's current question:
{question}

Respond only with a single, refined question that you will use to search the Knowledge Base.
It should be a VERY short specific question most likely to surface content. Focus on the question details.
Don't mention the company name unless it's a general question about the company.
IMPORTANT: Respond ONLY with the knowledgebase query, nothing else.
"""
    response = completion(model=MODEL, messages=[{"role": "system", "content": message}])
    return response.choices[0].message.content

In [42]:
rewrite_query("Who won the IIOTY award?", [])

'Who is the recent IIOTY award winner?'

In [56]:
def answer_question(question: str, history: list[dict] = []) -> tuple[str, list]:
    """
    Answer a question using RAG and return the answer and the retrieved context
    """
    query = rewrite_query(question, history)
    print(query)
    chunks = fetch_context(query)
    messages = make_rag_messages(question, history, chunks)
    response = completion(model=MODEL, messages=messages)
    return response.choices[0].message.content, chunks

In [57]:
answer_question("Who won the IIOTY award?", [])

Who was the recipient of the IIOTY award?
[3, 2, 1, 4, 5, 7, 8, 11, 13, 17, 19, 9, 12, 6, 14, 10, 15, 16, 18, 20]


("I don't have information about the winner of the IIOTY award in the provided data.",
 [Result(page_content='Career Progression at Insurellm (Part 2) and Performance 2022-2023\n\nThis section continues Oliver\'s career progression, his leading role in the \n\n## Insurellm Career Progression (continued)\n- **September 2022**: Assigned as the lead engineer for the new "Innovate" initiative, aimed at integrating AI-driven solutions into existing products.\n- **January 2023**: Awarded a mentorship role to guide new hires in backend technology and best practices within Insurellm.\n\n## Annual Performance History\n- **2018**: **3/5** - Adaptable team player but still learning to take initiative.\n- **2019**: **4/5** - Demonstrated strong problem-solving skills, outstanding contribution on the claims project.\n- **2020**: **2/5** - Struggled with time management; fell behind on deadlines during a high-traffic release period.\n- **2021**: **4/5** - Made a significant turnaround with organized

In [54]:
answer_question("Who went to Manchester University?", [])

Who went to Manchester University?
[8, 6, 14, 4, 16, 1, 10, 11, 12, 13, 15, 17, 18, 19, 20, 2, 3, 5, 7, 9]


('I do not have information indicating that any of the employees at Insurellm attended Manchester University.',
 [Result(page_content='HR Record Overview for Marcus Johnson\n\nThis section introduces Marcus Johnson, detailing his HR record including education, certifications, recognition, and mentorship contributions.\n\n# HR Record\n\n# Marcus Johnson\n\n## Summary\n- **Date of Birth:** February 17, 1988\n- **Job Title:** Customer Success Manager\n- **Location:** New York, New York\n- **Current Salary:** $98,000\n\n## Insurellm Career Progression\n- **April 2020 - Present:** Customer Success Manager\n  - Manages portfolio of 25 enterprise clients generating $5M in annual revenue\n  - Achieved 95% client retention rate over past three years\n  - Leads quarterly business reviews and strategic planning sessions with clients\n\n- **September 2018 - March 2020:** Customer Success Associate\n  - Supported senior CSMs with client onboarding and training\n  - Managed day-to-day communication 

As seen above, Query Re Writing can sometimes be very unreliable and helps sometimes